Notebook for developing the colour map & axis lock logic

In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

from typing import List

import pandas as pd
import numpy as np
from pof import Component, FailureMode
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pof.data.asset_data import SimpleFleet
import copy
from pof.loader.asset_model_loader import AssetModelLoader

In [2]:
from pof.paths import Paths

# Forecast years
START_YEAR = 2015
END_YEAR = 2024
CURRENT_YEAR = 2020

paths = Paths()

# Population Data
file_path = paths.input_path + os.sep
FILE_NAME = r"population_summary.csv"

sfd = SimpleFleet(file_path + FILE_NAME)
sfd.load()
sfd.calc_age_forecast(START_YEAR, END_YEAR, CURRENT_YEAR);

In [3]:
comp = Component.demo()
df = comp.expected_sensitivity(
        var_id="pole-fm-termites-dists-untreated-alpha", lower=0, upper=10, step_size=1, n_iterations=100, t_end=100
    );

100%|██████████| 100/100 [00:02<00:00, 34.07it/s]


In [4]:
def df_order(df, column):
    """
    sorts the dataframes for the graphs with total, risk and direct first
    """
    if column is None:
        raise ValueError("Column must be defined")

    if column == "task":
        values = df["task"].unique().tolist()
        values.sort()
    elif column == "source":
        values = df["source"].unique().tolist()
        values.sort()

    start_order = ["total", "inspection", "direct"]
    set_order = []

    for var in start_order:
        if var in values:
            set_order.append(var)

    for var in values:
        if var not in set_order:
            set_order.append(var)

    return_order = {}
    i = 1
    for var in set_order:
        return_order[var] = i
        i = i + 1
    # print(return_order)
    df_ordered = df.sort_values(by=[column], key=lambda x: x.map(return_order))
    # print(df_ordered.head())

    return df_ordered

In [5]:
def get_color_map(df, column):

    df = df_order(df, column)

    if column == "source":
        colors = px.colors.qualitative.Plotly
    elif column == "task":
        colors = px.colors.qualitative.Bold
    else:
        colors = px.colors.qualitative.Safe

    color_map = dict(zip(df[column].unique(), colors))

    return color_map

In [6]:
task_df = comp.expected_risk_cost_df(t_end=None)

task_df.head();

In [7]:
get_color_map(df=task_df, column="task");

#AXIS LOCK

In [8]:
aml = AssetModelLoader(paths.demo_path + os.sep + "Asset Model - Pole - Timber.xlsx")
comp_data = aml.load(paths.demo_path + os.sep + "Asset Model - Pole - Timber.xlsx")
comp = Component.from_dict(comp_data["pole"])
comp.fleet_data = sfd

t_end = 100

pof_sim = copy.copy(comp)
sens_sim = copy.deepcopy(comp)

# Complete the simulations
pof_sim.mp_timeline(t_end=t_end, n_iterations=1);

# Produce reports
pof_sim.expected_risk_cost_df(t_end=t_end);
# pof_sim.calc_pof_df(t_end=t_end)
pof_sim.calc_df_task_forecast(sfd.df_age_forecast);
# pof_sim.calc_summary(sfd.df_age)
pof_sim.calc_df_cond(t_end=t_end);


100%|██████████| 1/1 [00:00<00:00, 100.00it/s]


In [9]:
def get_y_max(chart, t_end=None, x_axis=None, y_axis=None, axis_lock=None):
    """ Determine the maximum y value for a given axis """

    global pof_sim
    global sens_sim

    if "pof" in chart:
        df = pof_sim.df_pof
        x_col = None
        y_col = "pof"
    elif "cond" in chart:
        df = pof_sim.df_cond
        x_col = None
        y_col = "y" + chart.split("_")[-1]
    elif "ms" in chart:
        df = pof_sim.df_erc
        x_col = "time"
        y_col = y_axis
    elif "sens" in chart:
        df = sens_sim.df_sens
        x_col = x_axis.split("-")[-1]
        y_col = y_axis
    elif "task" in chart:
        df = pof_sim.df_task
        x_col = None
        y_col = "pop_quantity"

    try:
        if axis_lock is None:
            if x_col is not None:
                y_max = df.groupby(x_col)[y_col].sum().max() * 1.05
            else:
                y_max = df[y_col].max() * 1.05
        else:
            ctx = dash.callback_context
            dash_id = ctx.triggered[0]["prop_id"].split(".")[0]
            if dash_id == "sens_var_y-dropdown" and x_col is not None:
                y_max = df.groupby(x_col)[y_col].sum().max() * 1.05
            else:
                return dash.no_update
    except:
        y_max = None
    return y_max


In [10]:
y_max_values = []
chart_list = [
    # "pof_fig",
    "ms_fig", 
    "sens_fig", 
    "task_fig"
]

for chart in chart_list:
    y_max_values.append(get_y_max(chart=chart, t_end=200, x_axis="cost", y_axis="cost", axis_lock=False))

y_max_values;

Divide by 0 - inf error logic

In [11]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

from pof.indicator import Indicator
from pof.component import Component
import pandas as pd
import scipy.stats as ss

comp = Component.demo()

In [12]:
comp.mp_timeline(t_end=100)

100%|██████████| 10/10 [00:00<00:00, 25.25it/s]


In [13]:
def _expected_condition(ec, conf):
    """
    Returns the expected condition based
    """
    # TODO make work for all condition levels loss:bool=False

    mean = ec.mean(axis=0)
    sigma = ec.std(axis=0)

    # Create a dataframe with mean & sigma
    df_mean_sigma = pd.DataFrame(
        data={"mean": mean, "sigma": sigma},
        columns=["mean", "sigma"],
    )

    # Filter out rows when sigma = 0
    # df_mean_sigma_filtered = df_mean_sigma[df_mean_sigma["sigma"] != 0]
    mean_filtered = df_mean_sigma[df_mean_sigma["sigma"] != 0]["mean"]
    sigma_filtered = df_mean_sigma[df_mean_sigma["sigma"] != 0]["sigma"]

    # TODO maybe add np.sqr(len(ec)) to make it stderr

    # Calculate bounds
    df_mean_sigma.loc[df_mean_sigma["sigma"] != 0, "upper"] = ss.norm.ppf(
        (1 - (1 - conf) / 2), loc=mean_filtered, scale=sigma_filtered
    )
    df_mean_sigma.loc[df_mean_sigma["sigma"] != 0, "lower"] = ss.norm.ppf(
        ((1 - conf) / 2), loc=mean_filtered, scale=sigma_filtered
    )
    df_mean_sigma.loc[df_mean_sigma["sigma"] == 0, "upper"] = df_mean_sigma["mean"]
    df_mean_sigma.loc[df_mean_sigma["sigma"] == 0, "lower"] = df_mean_sigma["mean"]


    # Adjust upper and lower to the mean if there is not variance (sigma was 0)
    # print(df_mean_sigma)
    # df_lower_upper = pd.merge(
    #     df_mean_sigma, df_mean_sigma_filtered, how="left", on=["mean", "sigma"]
    # )
    # df_lower_upper["upper"].fillna(df_lower_upper["mean"])
    # df_lower_upper["lower"].fillna(df_lower_upper["mean"])
    upper = df_mean_sigma["upper"]
    lower = df_mean_sigma["lower"]
    print(df_mean_sigma)

    # for i in range(0, len(upper)):
        # if self.decreasing:
        # if upper[i] > 100:
        #     upper[i] = 100
        # elif lower[i] < 0:
        #     lower[i] = 0
    upper[upper > 100] = 100
    lower[lower < 0] = 0
        # else:
        #     upper[upper[i] > 0] = 0
        #     lower[lower[i] < 100] = 100

    expected = dict(
        lower=lower,
        mean=mean,
        upper=upper,
    )

    return expected

In [16]:
for key,val in comp.indicator.items():
    ec = val.agg_timelines();

# _expected_condition(ec, 0.5);